In [5]:
import pandas as pd
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import smote_variants as sv
from tqdm import tqdm
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import os
import spacy as sp
from collections import Counter
import en_core_web_sm
import re

In [2]:
df=pd.read_csv('dataset/crawl_data.csv')
df

,body,description,title,url
0,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cocaine Drugs,http://fc3ryhftqfgwyroq7pt63f7jif4jknfrmd3pbdw...
1,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cannabis,http://r3q6nxpmfdxhzz3oumaseyfr5qujvg4cayzvj2y...
2,\n \n \n \n \n \n \n \n \n \n \n drugshop@dnmx...,"Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.","Buy Drugs - Ecstasy, LSD, Meth, Cocaine, Canna...",http://7nroowrqwdgbyajwrynxtypkdi46f22akwki73e...
3,\n \n \n \n \n \n \n \n \n \n \n drugsshopper...,Drugs Shop - buy cannabis best marijuana hemp.,Cannabis - Seeds Cannabis - Drugs Shop hash,http://hpcphv5q3lhiebh6c765lslihne4tn6ui5dnd63...
4,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",DRUGS,http://lnn6isw6tkbmplxe7eisjpf7nw6mmbwc7wjjuyh...
...,...,...,...,...
20245,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,"On the Internet, Forever-Mutating Debates\n",https://archive.nytimesn7cgmftshazwhfgzm37qxb4...
20246,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,1997 Technology Highlights,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...
20247,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Stock Volatility,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...
20248,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Wall Street,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...


In [3]:
sp.prefer_gpu()
nlp = en_core_web_sm.load()
def clean_text(doc):
    '''
    Clean the document. Remove pronouns, stopwords, lemmatize the words and lowercase them
    '''
    doc = nlp(doc)
    tokens = []
    exclusion_list = ["nan"]
    for token in doc:
        if token.is_stop or token.is_punct or token.text.isnumeric() or (token.text.isalnum()==False) or token.text in exclusion_list :
            continue
        token = str(token.lemma_.lower().strip())
        tokens.append(token)
    return " ".join(tokens) 

C:\Users\hetul\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [9]:
df["cleaned_website_text"]  = df.apply(lambda row: clean_text(row["body"]) ,axis=1)
df

,body,description,title,url,cleaned_website_text
0,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cocaine Drugs,http://fc3ryhftqfgwyroq7pt63f7jif4jknfrmd3pbdw...,cookie javascript trace protect privacy home q...
1,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",Cannabis,http://r3q6nxpmfdxhzz3oumaseyfr5qujvg4cayzvj2y...,cookie javascript trace protect privacy home q...
2,\n \n \n \n \n \n \n \n \n \n \n drugshop@dnmx...,"Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.","Buy Drugs - Ecstasy, LSD, Meth, Cocaine, Canna...",http://7nroowrqwdgbyajwrynxtypkdi46f22akwki73e...,home catalog question answer cart contact escr...
3,\n \n \n \n \n \n \n \n \n \n \n drugsshopper...,Drugs Shop - buy cannabis best marijuana hemp.,Cannabis - Seeds Cannabis - Drugs Shop hash,http://hpcphv5q3lhiebh6c765lslihne4tn6ui5dnd63...,home catalog question answer cart contact cate...
4,"\n \n \n \n \n \n No cookies, no javascript, n...","Drugs Store - buy Ecstasy, Cocaine, LSD, Meth.",DRUGS,http://lnn6isw6tkbmplxe7eisjpf7nw6mmbwc7wjjuyh...,cookie javascript trace protect privacy home q...
...,...,...,...,...,...
20245,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,"On the Internet, Forever-Mutating Debates\n",https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,january internet forever mutating debates rela...
20246,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,1997 Technology Highlights,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,january technology highlights related articles...
20247,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Stock Volatility,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,credit new york times article home site index ...
20248,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \...,NaN,Wall Street,https://archive.nytimesn7cgmftshazwhfgzm37qxb4...,volatile day volatile year wall street clean n...


In [10]:
df.to_csv("crawledcleaned.csv")